## **Turi Create Object Detection**
The following notebook includes an example of data import, model training and model saving for an object detection model.

In [2]:
%%capture
import urllib.request
import tarfile
import turicreate as tc
#from skafossdk import *
import save_models as sm
import coremltools

In [2]:
#ska = Skafos()

## **The Data**
The data for this example are 785 images which breaks down to 420 cars and 365 bikes and we have loaded the cleaned data to a public S3 bucket. The data cleaning process and can be found in the `load_object_data.ipynb`

In [3]:
data_url = "https://s3.amazonaws.com/skafos.example.data/ig02.sframe.tar.gz"
data_path = "ig02.sframe.tar.gz"

# pull the tar
#ska.log("Retrieving the images from online", labels = ['image_similarity'])
retrieve = urllib.request.urlretrieve(data_url, data_path)

# extract the file
#ska.log("Images downloaded, extracting the images", labels = ['image_similarity'])
tar = tarfile.open(data_path)
tar.extractall()
tar.close()

In [4]:
# Load the data
data =  tc.SFrame('ig02.sframe')

In [7]:
data.shape

(785, 4)

## **Data Load**

In [ ]:
# Load the data
data =  tc.SFrame('ig02.sframe')

# Make a train-test split
train_data, test_data = data.random_split(0.8)

## **Model Build and Save**
- Build the model by passing the training data to the turi create function.
- Specify # of iterations due to runtime concerns.
- Save the model by:
    - exporting to coreml
    - converting the weights to half-precision to decrease model size
    - compress the model
    - save the model to Skafos

In [ ]:
# Create a model
model = tc.object_detector.create(train_data, max_iterations=900)

In [ ]:
# Save the model for later use in Turi Create
coreml_model_name = 'object_detection.mlmodel'
res = model.export_coreml(coreml_model_name)

# Convert to half-precision
model_spec = coremltools.utils.load_spec(coreml_model_name)
model_fp16_spec = coremltools.utils.convert_neural_network_spec_weights_to_fp16(model_spec)
coremltools.utils.save_spec(model_fp16_spec, coreml_model_name)

# compress the model
compressed_model_name, compressed_model = sm.compress_model(coreml_model_name)

# save to Skafos
sm.skafos_save_model(skafos = ska, model_name = compressed_model_name,
								compressed_model = compressed_model,
								permissions = 'public')